In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import openai
import json
import ijson
import requests
from pywikidata import Entity
import os
openai.api_key = os.getenv('KEY')

openai.base_url = "https://api.vsegpt.ru:6070/v1/"

/Users/aa/vmk/diplom/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [14]:
file_path = "lol/RuBQ_2.0_test.json"
with open(file_path, 'r') as file:
    array_items = ijson.items(file, 'item')
    
    i = 0
    true_answers = 0
    answers = []
    for item in array_items:
        messages = []
        i += 1

        question = item["question_text"]
        answer = item["answer_text"].lower()
        # prompt = "Answer shortly in 1-3 words. If answer is a number, print digits"
        prompt = "Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже."
        # messages.append({"role": "user", "content": prompt})
        messages.append({"role": "user", "content": prompt + ' ' + question})
        response = ""
        print(messages)
        try:
            response_big = openai.chat.completions.create(
                model="openai/gpt-3.5-turbo-1106",
                messages=messages,
                temperature=0.1,
            )
            response = response_big.choices[0].message.content.lower()
            print(response)
        except Exception as e:
            print(e)
            response = ""
            
        answers.append({
            "question": question,
            "answer": response,
            "correct_answer": answer,
            "is_correct": response in answer or answer in response
        })
        print(response, answer)
        if response in answer or answer in response:
            true_answers += 1
        print(i, true_answers)
        json_data = json.dumps(answers, indent=4)
        with open("questions_answers_rubq_ru.json", "w", encoding="utf-8") as json_file:
            json_file.write(json_data)

[{'role': 'user', 'content': 'Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже. Что может вызвать цунами?'}]
подводное землетрясение.
подводное землетрясение. землетрясение
1 1
[{'role': 'user', 'content': 'Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже. Кто написал роман «Хижина дяди Тома»?'}]
харриет бичер стоу.
харриет бичер стоу. г. бичер-стоу
2 1
[{'role': 'user', 'content': 'Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже. Кто автор пьесы «Ромео и Джульетта»?'}]
уильям шекспир.
уильям шекспир. шекспир
3 2
[{'role': 'user', 'content': 'Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже. Как называется столица Румынии?'}]
бухарест.
бухарест. бухарест
4 3
[{'role': 'user', 'content': 'Ответь максимально коротко и ёмко на вопрос. Ответ приводи в именительном падеже. На каком инструменте играл Джимми Хендрикс?'}]
гитара.
гитара. гитара
5 4
[{'role': 'user', 

In [9]:
def send_request(question_text):
    url = 'http://5.35.10.12:8080/pipeline/act_selection/main'
    headers = {'Content-Type': 'application/json'}
    data = {'text': question_text}

    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [10]:
response = send_request("Largest country in the world?")
response

{'answers': ['Q159',
  'Q15180',
  'Q878',
  'Q142',
  'Q27',
  'Q145',
  'Q212',
  'Q232',
  'Q183',
  'Q16',
  'Q414',
  'Q865',
  'Q218',
  'Q31',
  'Q55',
  'Q29',
  'Q38',
  'Q36',
  'Q43',
  'Q843',
  'Q794',
  'Q884',
  'Q928',
  'Q45'],
 'instance_of_score': [0.7727272727272727,
  0.9545454545454546,
  0.9090909090909091,
  0.8636363636363636,
  0.8181818181818182,
  0.7727272727272727,
  0.9545454545454546,
  0.8181818181818182,
  0.9090909090909091,
  0.9090909090909091,
  0.7727272727272727,
  0.8181818181818182,
  0.9090909090909091,
  0.7727272727272727,
  0.9090909090909091,
  0.8181818181818182,
  0.9090909090909091,
  0.8636363636363636,
  0.7272727272727273,
  0.9090909090909091,
  0.8636363636363636,
  0.8181818181818182,
  0.8636363636363636,
  0.8181818181818182],
 'forward_one_hop_neighbours_score': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],


In [12]:
file_path = "lol/RuBQ_2.0_test.json"
with open(file_path, 'r') as file:
    array_items = ijson.items(file, 'item')
    
    i = 0
    true_answers = 0
    answers = []
    for j, item in enumerate(array_items):
        messages = []
        i += 1

        question = item["question_text"]
        answer = item['answer_text']
        response_data = send_request(question)
        response = ""
        if not response_data:
            response = "Error"
        else:
            try:
                response = Entity(response_data['answers'][0]).label.lower()
            except Exception as e:
                response = "Error"
        answers.append({
            "question": question,
            "answer": response,
            "correct_answer": answer.lower(),
            "is_correct": response in answer.lower()
        })
        if response in answer.lower():
            true_answers += 1
        print(i, true_answers)
        json_data = json.dumps(answers[len(answers) - 1], indent=4, ensure_ascii=False)
        with open("questions_answers_rubq_act_ru.json", "a") as json_file:
            json_file.write(json_data + ",\n")

1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
Error: 500
11 0
12 0


KeyboardInterrupt: 